In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import sys
from pyspark.sql import Catalog

In [2]:
spark = SparkSession \
    .builder \
    .appName("Car Sales Report") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")\
    .config("spark.sql.catalog.spark_catalog.type", "hive")\
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")\
    .config("spark.yarn.access.hadoopFileSystems", "s3a://go01-demo/")\
    .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [4]:
#spark.conf.get("spark.jars")

In [5]:
data_lake_name = "s3a://go01-demo/"
s3BucketName = "s3a://go01-demo/cde-workshop/cardata-csv/"
# Your Username Here:
username = "test_user_112222_7"

In [6]:
spark.catalog.currentDatabase()

'default'

In [7]:
df = spark.read.option("inferSchema", "true").option("header", "true").csv("/home/cdsw/data/batch_data_1.csv")

In [8]:
df.show()

+------------------+--------------------+-------------------+--------+--------------------+--------------------+-------+-------+-------------+---------+---------+-----------+------------+---------------+------------------+
|              name|      street_address|               city|postcode|        phone_number|                 job|recency|history|used_discount|used_bogo| zip_code|is_referral|     channel|          offer|             score|
+------------------+--------------------+-------------------+--------+--------------------+--------------------+-------+-------+-------------+---------+---------+-----------+------------+---------------+------------------+
|      Austin Duran|     5204 Emily Club|     Lake Mollyside|   79396| +1-070-516-3006x662|   Film/video editor|      5| 171.13|            0|        1|    Urban|          0|       Phone|Buy One Get One|1.0335466196578111|
|      Austin Smith| 97370 William Vista|        Lake Nicole|   98418|     +1-901-430-6901| Corporate treasu

In [11]:
df.dtypes

[('name', 'string'),
 ('street_address', 'string'),
 ('city', 'string'),
 ('postcode', 'int'),
 ('phone_number', 'string'),
 ('job', 'string'),
 ('recency', 'int'),
 ('history', 'double'),
 ('used_discount', 'int'),
 ('used_bogo', 'int'),
 ('zip_code', 'string'),
 ('is_referral', 'int'),
 ('channel', 'string'),
 ('offer', 'string'),
 ('score', 'double')]

In [21]:
spark.sql("DROP TABLE IF EXISTS default.batch_data_test")

DataFrame[]

In [22]:
spark.sql("CREATE TABLE IF NOT EXISTS spark_catalog.default.batch_data_test (name string, street_address string, city string, postcode int, phone_number string, job string, recency int, history double, used_discount int, used_bogo int, zip_code string, is_referral int, channel string, offer string, score double) USING iceberg")

DataFrame[]

In [28]:
df.writeTo("spark_catalog.default.icetest").using("iceberg").createOrReplace()
#df.writeTo("spark_catalog.default.batch_data_test").append()

23/01/06 01:03:14 498 ERROR TaskSetManager: Task 0 in stage 9.0 failed 4 times; aborting job
23/01/06 01:03:14 500 ERROR AtomicReplaceTableAsSelectExec: Data source write support IcebergBatchWrite(table=default.icetest, format=PARQUET) is aborting.
23/01/06 01:03:14 500 ERROR AtomicReplaceTableAsSelectExec: Data source write support IcebergBatchWrite(table=default.icetest, format=PARQUET) aborted.
23/01/06 01:03:14 500 ERROR Utils: Aborting task
org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:613)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:389)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:333)
	at org.apache.spark.sql.execution.datasources.v2.AtomicReplaceTableAsSelectExec.writeWithV2(WriteToDataSourceV2Exec.scala:193)
	at org.apache.spark.sql.execut

Py4JJavaError: An error occurred while calling o169.createOrReplace.
: org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:613)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:389)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:333)
	at org.apache.spark.sql.execution.datasources.v2.AtomicReplaceTableAsSelectExec.writeWithV2(WriteToDataSourceV2Exec.scala:193)
	at org.apache.spark.sql.execution.datasources.v2.TableWriteExecHelper.$anonfun$writeToTable$1(WriteToDataSourceV2Exec.scala:483)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1496)
	at org.apache.spark.sql.execution.datasources.v2.TableWriteExecHelper.writeToTable(WriteToDataSourceV2Exec.scala:471)
	at org.apache.spark.sql.execution.datasources.v2.TableWriteExecHelper.w

In [24]:
df.write.mode("overwrite").saveAsTable('default.iceberg_example_010523'.format(username), format="parquet")

In [25]:
spark.sql("ALTER TABLE default.iceberg_example_010523 UNSET TBLPROPERTIES ('TRANSLATED_TO_EXTERNAL')")

DataFrame[]

In [26]:
spark.sql("CALL spark_catalog.system.migrate('default.iceberg_example_010523')")

DataFrame[migrated_files_count: bigint]

In [27]:
spark.sql("SELECT * FROM spark_catalog.default.iceberg_example_010523.PARTITIONS").show()

23/01/06 00:52:34 683 ERROR TaskSetManager: Task 0 in stage 8.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o166.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 4 times, most recent failure: Lost task 0.3 in stage 8.0 (TID 19) (100.100.82.184 executor 4): java.lang.ClassNotFoundException: org.apache.iceberg.spark.source.SparkScan$ReadTask
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.apache.spark.serializer.JavaDeserializationStream$$anon$1.resolveClass(JavaSerializer.scala:68)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1988)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1852)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2186)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.ja

In [12]:
spark.catalog.listTables('default')

[Table(name='t', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='s', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='lc_smote_subset', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='driver_stats', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='table1', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='ttable2', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employee_ofer', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='tax_auth_heb', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='results_anomaly', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='mytable', database='default', description=None, 

In [32]:
#---------------------------------------------------
#               LOAD ICEBERG TABLES AS DATAFRAMES
#---------------------------------------------------
spark.sql("SELECT * FROM spark_catalog.default.batchdataiceberg").show()

+-------------------+---------+-----------+-----------+---------------+--------------------+
|    event_timestamp|driver_id|  conv_rate|   acc_rate|avg_daily_trips|             created|
+-------------------+---------+-----------+-----------+---------------+--------------------+
|2022-12-06 08:00:00|     1003| 0.74933994|  0.9661571|            246|2022-12-21 08:33:...|
|2022-12-06 09:00:00|     1003|  0.4900406| 0.79025567|            629|2022-12-21 08:33:...|
|2022-12-06 10:00:00|     1003| 0.44497514|  0.7003286|            409|2022-12-21 08:33:...|
|2022-12-06 11:00:00|     1003| 0.83675057|0.023251861|            301|2022-12-21 08:33:...|
|2022-12-06 12:00:00|     1003| 0.60384226|  0.8531334|            159|2022-12-21 08:33:...|
|2022-12-06 13:00:00|     1003|  0.3236954| 0.18340571|              1|2022-12-21 08:33:...|
|2022-12-06 14:00:00|     1003|  0.0521478| 0.30485526|            293|2022-12-21 08:33:...|
|2022-12-06 15:00:00|     1003|  0.9964056| 0.36764374|            908

In [22]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [23]:
#spark.sql("USE spark_catalog.{}_CAR_DATA".format(username))

In [24]:
#spark.sql("SHOW CURRENT NAMESPACE").show()

In [25]:
#spark.sql("SELECT * FROM spark_catalog.{}_CAR_DATA.CAR_SALES.history".format(username)).show()

In [58]:
# ICEBERG TABLE HISTORY (SHOWS EACH SNAPSHOT AND TIMESTAMP)
print("SHOW ICEBERG TABLE HISTORY")
#print("SELECT * FROM spark_catalog.default.table_pessoa.history;")
spark.sql("SELECT * FROM spark_catalog.default.batchdataiceberg.history").show()

SHOW ICEBERG TABLE HISTORY


AnalysisException: spark_catalog requires a single-part namespace, but got [default, batchdata]

In [8]:
def create_job_body(*runtime_addon_ids):
    print(type(runtime_addon_ids))
    print(runtime_addon_ids)
    ls = list(runtime_addon_ids)
    print(ls)

In [10]:
create_job_body("yoyo", "yeh", "haha")

<class 'tuple'>
('yoyo', 'yeh', 'haha')
['yoyo', 'yeh', 'haha']
